# 01장 Getting Started with Spark and GraphX

### Spark와 GraphX는 무엇인가?
- 거대한 분산된 데이터셋을 처리하기 위한 클러스트 기반 컴퓨팅 플랫폼임.
- 최적화된 병렬 컴퓨팅 엔진과  유연하고 단일한 API을 제공하여  빠르고 쉽게 데이터셋을 처리함.
- Spark의 핵심 추상화 개념은 Resilient Distributed Dataset (RDD)을 기본 개념으로 함.
- MapReduce framework을 확장하여 Spark Core API는 더욱 쉽게 데이터 분석 작업을 할 수 있음.
- Spark는 머신러닝과 그래프 프로세싱과 같은 특별한 하이 레벨 Library을 제공함.
- GraphX는 그래프 병렬 처리를 수행하는 Library임.

### 이번장의 목표
- Spark을 설치하기
- Spark shell을 실험하고  Spark의 데이터 추상화에 대해서 알아보기
- 기본 RDD과 graph 연산자들을  가지고 links들을 생성하고 탐색하기
- SBT로 Spark 어플을 빌드 및 실행

## 01절. Downloading and installing Spark 1.4.1

### Java Development Kit 7 (JDK) 설치
- http://www.oracle.com/technetwork/java/javase/downloads/jdk7-downloads-1880260.html 

### download the latest release of Spark
- https://spark.apache.org/downloads.html
- Pre-built for Hadoop 2.6 and later과 Direct Download을 선택
- wget http://apache.mirror.cdnetworks.com/spark/spark-1.5.2/spark-1.5.2-bin-hadoop2.6.tgz
- tar xvf spark-1.5.2-bin-hadoop2.6.tgz
- ln -s spark-1.5.2-bin-hadoop2.6 spark
- cd spark 
- ls 

### Spark의 내부 디렉토리 구조
- core : sparkAPI와 핵심 구성요소의 소스코드가 포함됨.
- bin : spark app과 spark shell을 실행시키기 위한 실행파일들이 포함됨.
- graphx, mllib, sql, streaming : graph processing, machine learning, queries, and stream processing에 관련된 LIbrary
- example : spark app의 예제와 데모를 포함.
- conf : slave 노드와 기타 설정파일이 포함.

### Spark 경로 설정
- ~/.bashrc  or ~/.bash_profile
```
export SPARKHOME="/home/deepbio/app/spark/"
export SPARKSCALAEX="/home/deepbio/app/spark/examples/src/main/scala/org/apache/spark/examples/"
```

- source ~/.bashrc or ~/.bash_profile
- ls -al  $SPARKSCALAEX/graphx/LiveJournalPageRank.scala  # 경로 설정을 확인함.

## 02절 Experimenting with the Spark shell

- Scala 와 python을 위한 2가지 shell을 지원
- GraphX는 대부분은 scala에서 제대로 동작함.
- 실행방법

```
$SPARKHOME/bin/spark-shell  --driver-memory 2g

cd $SPARKHOME
./bin/spark-shell   
```

- 아래와 같은 에러가 발생하면 컴파일 안 된 소스만 받은 상태임.  prebuilt version 으로  다운로드 받음.
```
Failed to
find Spark assembly in spark-1.4.1/assembly/target/
scala-2.10. You need to build Spark before running
this program,
```

- spark의 정상동작 확인
```
scala> sc
scala> val myRDD = sc.parallelize(List(1,2,3,4,5))
scala> sc.textFile("README.md").filter(line => line contains "Spark").count()
```

- sc : Spark context is the point of entry to the Spark API
- parallelize()함수를 호출해서 RDD 객체를 생성
- README.md 파일을 로드해서 "spark" 라는 단어가 포함된 라인을 필터링하고  마지막으로 해당 라인의 수를 출력함.
- spark의 기본적은 RDD 변환 및 action은 아래 URL을 참고함.
```
https://spark.apache.org/docs/latest/programming-guide.html
```


## 03절 Getting started with GraphX

#### 이번절의 목표
- 1) 구체적인 예제를 통해서 Spark Core API와 GraphX API를 사용해서 graph를 생성하고 탐색하는 방법을 배움.
- 2) graph processing을 하는데 중요한 스칼라 프로그램 특정을 알아봄.
- 3) standalone Spark app을 개발하고 실행해봄.

### Building a tiny social network
- GraphX와 RDD 모듈을 import 

```
scala> import org.apache.spark.graphx._
scala> import org.apache.spark.rdd.RDD
```

### Loading the data
- $SPARKHOME/data/ 디렉토리안의 people.csv와 links.csv 파일을 생성시킴.
- people.csv의 내용
```
1,Alice,20
2,Bob,18
3,Charlie,30
4,Dave,25
5,Eve,30
6,Faith,21
7,George,34
8,Harvey,47
9,Ivy,21
```

- links.csv의 내용
```
1,2,friend
1,3,sister
3,2,boss
2,4,brother
4,5,client
6,7,cousin
7,9,coworker
8,9,father
1,9,friend
```


```
scala> val people = sc.textFile("./data/people.csv")
scala> val links = sc.textFile("./data/links.csv")
```

- vertices와 edges의 알맞는 자료구조 변환하기 위해서 string 파싱이 필요함.

### The property graph
- spark에서  graph을 정의하는 자료구조
- 방향성 멀티 그래프
    - 한 쌍의 vertices에는 여러개의 edges를 가질 수 있음.
    - 각각의 edge는 방향성과 비방향성 관계를 가질 수 있음.
```
class Graph[VD, ED] {
val vertices: VertexRDD[VD]
val edges: EdgeRDD[ED,VD]
}
```

### Transforming RDDs to VertexRDD and EdgeRDD
- graph를 구축하기 위한 3개의 단계
    - 1) Persion 클래스 정의
```
case class Person(name: String, age: Int)
```

    - 2) people과 links 객체에 있는 csv 문자열을 파싱하고, Person 객체와 Edge 객체를 생성함.  각각의 결과는 RDD[(VertexId, Person)] 과  RDD[Edge[String]] 컬렉션안에 포함됨.

```
scala> val peopleRDD: RDD[(VertexId, Person)] = people map { line =>
    val row = line split ','
    (row(0).toInt, Person(row(1), row(2).toInt))
}
scala> type Connection = String
scala> val linksRDD: RDD[Edge[Connection]] = links map {line =>
val row = line split ','
Edge(row(0).toInt, row(1).toInt, row(2))
}

```
     - 3) Graph()의 생성자를 가지고 tinySocial 라는 이름을 갖는  social graph를 생성함.
```
scala> val tinySocial: Graph[Person, Connection] = Graph(peopleRDD, linksRDD)
```    

- Graph는 VertexRDD[VD]와 EdgeRDD[ED,VD]의 인스턴스를 멤버로 갖지만, 위의 코드에서는 RDD[(VertexId, Person)]와  RDD[Edge[Connection]] 을 가지고 생성자에 넘겨줌.
- 이것이 가능한 이유는 => VertexRDD[VD]와 EdgeRDD[ED,VD]는 RDD[(VertexId, Person)] and RDD[Edge[Connection]]의 서브클래스로 동작함.    좀더 구체적인 이유는 나중에 설명함.

### Introducing graph operations
- vertices and edges을 내용물들을 확인해봄.
```
scala> tinySocial.vertices.collect()
scala> tinySocial.edges.collect()
```

- professional 연결을 출력하기 위해서 profLinks 라는 리스트를 생성함.
```
scala> val profLinks: List[Connection] = List("coworker", "boss", "employee","client", "supplier")
```

```
val profNetwork = tinySocial.edges.filter{ 
    case Edge(_,_,link) => profLinks.contains(link)
 }
for {
    Edge(src, dst, link) <- profNetwork.collect()
    srcName = (peopleRDD.filter{case (id, person) => id == src} first)._2.name
    dstName = (peopleRDD.filter{case (id, person) => id == dst} first)._2.name
} println(srcName + " is a " + link + " of " + dstName)
```

- 위의 코드는 매우 비효율적임. 
- GraphX library 는 2가지 다른 view 방식을 제공
     - 1) edges, vertices의 graph 또는 테이블 
     - 2) triplets
- 각각의 view에는 최적화된 많은 연산자들이 제공됨.
```
scala> tinySocial.subgraph(profLinks contains _.attr).
    triplets.foreach(t => println(t.srcAttr.name + " is a " +
    t.attr + " of " + t.dstAttr.name))
```

- subgraph()함수는 professional links을 filter하는데 사용하고, triplet view는 edges 와 vertices의 속성을 접근할때 사용함.
- EdgeTriplet 은 3-tuple ((VertexId, Person), (VertexId, Person), Connection) 의 alias type임

## 04절. Building and submitting a standalone application

### Writing and configuring a Spark program
- simpleGraph.scala 파일을 만들고 아래 코드를 넣음.
- http://www.packtpub.com 에서 가서 예제를 다운로드 받을 수 있음.

In [ ]:
package com.github.giocode.graphxbook

import org.apache.spark.SparkContext
import org.apache.spark.SparkContext._
import org.apache.spark.SparkConf
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD

object SimpleGraphApp {
	def main(args: Array[String]){
		// Configure the program
		val conf = new SparkConf()
					.setAppName("Tiny Social")
					.setMaster("local")
					.set("spark.driver.memory", "2G")
		val sc = new SparkContext(conf)

		// Load some data into RDDs
		val people = sc.textFile("../../data/people.csv")
		val links = sc.textFile("../../data/links.csv")

		// Parse the csv files into new RDDs
		case class Person(name: String, age: Int)
		type Connexion = String
		val peopleRDD: RDD[(VertexId, Person)] = people map { line => 
			val row = line split ','
			(row(0).toInt, Person(row(1), row(2).toInt))
		}
		val linksRDD: RDD[Edge[Connexion]] = links map {line => 
			val row = line split ','
			Edge(row(0).toInt, row(1).toInt, row(2))
		}

		// Create the social graph of people
		val tinySocial: Graph[Person, Connexion] = Graph(peopleRDD, linksRDD)
		tinySocial.cache()
		tinySocial.vertices.collect()
		tinySocial.edges.collect()

		// Extract links between coworkers and print their professional relationship
		val profLinks: List[Connexion] = List("coworker", "boss", "employee","client", "supplier")
		tinySocial.subgraph(profLinks contains _.attr).
				   triplets.foreach(t => println(t.srcAttr.name + " is a " + t.attr + " of " + t.dstAttr.name))
	}
}

### Building the program with the Scala Build Tool 

- http://www.scala-sbt.org/0.13/tutorial/index.html 에서 SBT 설치방법, 수동설치보다는 package manager로 설치를 추천함.
- 우분투에서 설치 방법
```
echo "deb https://dl.bintray.com/sbt/debian /" | sudo tee -a /etc/apt/sources.list.d/sbt.list
sudo apt-key adv --keyserver hkp://keyserver.ubuntu.com:80 --recv 642AC823
sudo apt-get update
sudo apt-get install sbt
```

- CentOS에서 설치 방법
```
curl https://bintray.com/sbt/rpm/rpm | sudo tee /etc/yum.repos.d/bintray-sbt-rpm.repo
sudo yum install sbt
```

- build.sbt 파일 생성
```
name := "Simple Project"
version := "1.0"
scalaVersion := "2.10.4"
libraryDependencies ++= Seq(
    "org.apache.spark" %% "spark-core" % "1.4.1",
    "org.apache.spark" %% "spark-graphx" % "1.4.1"
)
resolvers += "Akka Repository" at "http://repo.akka.io/releases/"
```

- 빌드  : 
```
$ sbt package
```

- simple-project_2.10-1.0.jar 라는 jar 생성됨

### Deploying and running with spark-submit
- 아래와 같이 실행함.

```
cd target/scala-2.10/

../../bin/spark-submit --class \
  com.github.giocode.graphxbook.SimpleGraphApp \
 ./simple-project_2.10-1.0.jar
```